# CS231n Winter 2016: Lecture 9
## Topics
Visualization, Deep Dream, Neural Style, Adversarial Examples
**TODO:** add topics. hm maybe there is some plugin to do this quick?

## Sources
- video: https://www.youtube.com/watch?v=ta5fdaqDT3M

In [1]:
from IPython.display import HTML

def play_youtube(video_id):
    return HTML(f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')
play_youtube('ta5fdaqDT3M')

## visualize patches that maximally activate neurons
- pick arbitrary neuron on layer N(for example pool 5)
- pipe a lot of images and see which activates this neuron
- then we could visualize layer by putting the image which has miximum feedback on this neuron

## visualize weights
- visualizing raw weights works only for 1st layer
  - but here is hard to find something new and interesting. because almost always they are rotated [gabor like feature](https://en.wikipedia.org/wiki/Gabor_filter)
  - PCA could give sin/cos like images
  

## visualize representation space (e.g. with t-SNE)

[t-SNE visualization of CNN codes](https://cs.stanford.edu/people/karpathy/cnnembed/)
- take the last CNN layer immediately before classifier. 
- it's like be n-dimention "code" for image - summarize content of the image
- use t-SNE to embede it to 2d
- pass all images and find the place in 2d for it
- in result we will get map where nearest images have similar "code" in CNN and could be classified in the similar way

## occusion experiments

- slide patch of zeros over the image 
- look how probability of classifier change
- we get heat map of probability

## Visualizing activations

- tool: http://yosinski.com/deepvis


## deconv approaches (single backward pass)
researches
- [Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps Karen Simonyan, Andrea Vedaldi, Andrew Zisserman](https://arxiv.org/abs/1312.6034)
- [Striving for Simplicity: The All Convolutional Net Jost Tobias Springenberg, Alexey Dosovitskiy, Thomas Brox, Martin Riedmiller](https://arxiv.org/abs/1412.6806)

steps
- get one neuron
- set gradient for it all he rest will be 0
- back propagate its gradient back to the image
- we get reconstructed image
- use guided back prob (instead)
  - change backwarp for ReLU layer
  
  by default
  $$
  f_i^{l+1} = relu(f_i^l) = max(f_i^l, 0)
  $$
  $$
  R_i^l = (f_i^l > 0) R_i^{l+1},\ where\ R_i^{l+1} = \frac{\delta f^{out}}{\delta f_i^{l+1}}
  $$
  guide only possitive
  $$
  R_i^l = (f_i^l > 0) (R_i^{l+1} > 0) R_i^{l+1},\ where\ R_i^{l+1} = \frac{\delta f^{out}}{\delta f_i^{l+1}}
  $$
  we keep attention to the positive influences.
  
  **ME:** interesting what we get when we show only negative influences. Btw this question was in a lecture. so we could try
  
### DeConvNet
Gives much cleaner images.
Ignore relu
  $$
  R_i^l = (R_i^{l+1} > 0) R_i^{l+1},\ where\ R_i^{l+1} = \frac{\delta f^{out}}{\delta f_i^{l+1}}
  $$
but it works as well

## human experiment comparisons

## optimization over image approaches (optimization)

find an image that maximizes some class score
- start with blank/random image 
- pass it though network
- pick one class
- add positive gradient 
- propagate it back to the image
- update the image

use L2 regularization
$$
\arg \max_{I} S_c(I) - \lambda ||I||_2^2
$$

[Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps Karen Simonyan, Andrea Vedaldi, Andrew Zisserman](https://arxiv.org/abs/1312.6034)
- forward prop image
- find the class
- set positive gradient (1)
- back prob it to the image as heat map
- we get a heat map of pixels which could increase the most the score for the class

we could use `grabcut` for segmentation and crop out the most important images


[Understanding Neural Networks Through Deep Visualization Jason Yosinski, Jeff Clune, Anh Nguyen, Thomas Fuchs, Hod Lipson](https://arxiv.org/abs/1506.06579)
differen regularization - ignore the penalty:
$$
\arg \max_{I} S_c(I)
$$
- blure image between updates
- take any pixel with small norm to zero (to encourage sparsity)


as well we could try to visualize image for different layers


## Information of the last layer
[Understanding Deep Image Representations by Inverting Them Aravindh Mahendran, Andrea Vedaldi](https://arxiv.org/abs/1412.0035)

and could we reconstruct image based on the last layer?
$$
x^* = \arg \max_{x \in R^{H \times W \times C}} L(\Phi(x), \Phi_0) + \lambda R(x)
$$
$$
L(\Phi(x), \Phi_0) = \|\Phi(x) - \Phi_0\|^2
$$
and can do it for all layers, the higher we are the more abstruct result we get

continue here: https://youtu.be/ta5fdaqDT3M?t=2562

## DeepDream
[github.com/google/deepdream](https://github.com/google/deepdream)

amplify the features which activated in the network

- apply jittering normalization to the destanation image
- forward to the some layer
- apply gradient equal to the activation (objective layer)
- backward from that point
- apply normalized update
- apply jittering normalization to the destanation image

for example: when network detects a little bit dog face in a image we will bust it and highlight this (dreaming) dog phase.
it "dreams" dogs because it has a lot of dogs in training set: 1k classes and 100 is dogs.

In [2]:
play_youtube('DgPaCWJL7XI')

In [3]:
play_youtube('oyxSerkkP4o')

## Neural Style
- [A Neural Algorithm of Artistic Style Leon A. Gatys, Alexander S. Ecker, Matthias Bethge](https://arxiv.org/abs/1508.06576)
- https://github.com/jcjohnson/neural-style
- deepart.io

algorithm
- input: content image, style image
- content image:
    - pass content image forward to the CNN
    - store all activations - they are correspond to content of the image

- style:
    - pass forward to the CNN
    - get pairwise statistics of activations in N layer (covariance matrixes)
    - how often each pair of features come together
        - get d dimension fiber from 3d matrix `w x h * d` 
          - we want statistic (covariance or Gram matrix) of those products - outer product and summed across spatial locations
        - in other words: reshape layer in 3d matrix to 2d `d x w * h`
        - use $G = V^TV$

- loss function
    - content loss (activations match content) (usually 1 layer)
    - style loss (gram matrices of activations match style) (many layers)
continue here: https://youtu.be/ta5fdaqDT3M?t=3218

- because we don't have a lot of data here it is excellent case for [Limited-memory BFGS ](https://en.wikipedia.org/wiki/Limited-memory_BFGS).
  - so we could use 2nd derivatives here

**ME:**
1. what if we do for content simetry modification as we have for style activation?
  - for style we try to match pairwise neurons and ignore spatial distribution (Gram matrix)
  - so could we ignore d dimension and just focus on distribution? 
    - maybe it will extract raw structure and ignore feature statistics which we have in the original image?
2. could we generate raw image - by removing any style from it? And how would it look like?
  - I could get few images with single style and try to extract:
    - style of each of image 
    - target image itself
    - or maybe summary style
3. Maybe it's the same as Adversarial examples?

## "Fool" ConvNet / Adversarial Examples
research
- [Intriguing properties of neural networks Christian Szegedy, Wojciech Zaremba, Ilya Sutskever, Joan Bruna, Dumitru Erhan, Ian Goodfellow, Rob Fergus](https://arxiv.org/abs/1312.6199) - small distorshen change class of the image (ostrich class)
- [Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images Anh Nguyen, Jason Yosinski, Jeff Clune](https://arxiv.org/abs/1412.1897) - some pattern could be class (but it doesn't look for us as this class
- [Exploring the representation capabilities of the HOG descriptor](http://ieeexplore.ieee.org/document/6130416/?reload=true) we have this problem before, for example for HOG
- [Explaining and Harnessing Adversarial Examples Ian J. Goodfellow, Jonathon Shlens, Christian Szegedy](https://arxiv.org/abs/1412.6572) - "primary cause of neural network" vulnerability to adveserial perburbation is their linear nature (which we use in the forward path). we train only on linear manifolds and ignore huge amount of others (they are in the "shadows" of classified manifolds)
- [Breaking Linear Classifiers on ImageNet by Andrej Karpathy](http://karpathy.github.io/2015/03/30/breaking-convnets/) - so it ever works for basic linear classier because we know where gradient leads us and we can change weights to any class we want

how to solve it
- get few crops of image
- train on adversarial examples
- replace linear function - but it will break generalization of the net
- but there is no final solution yet